<a href="https://colab.research.google.com/github/jakobatgithub/unreverb/blob/main/ddsp_reverb_t2_pof.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade pip
!pip install --upgrade ddsp
!pip install tensorflow_io

import tensorflow as tf

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
sample_length = 16000
# load data
# path of the tensorflow dataset 
path = "/content/drive/My Drive/dsr_project/data/but-czas_v1.0/wavs/"

# load the tensorflow dataset
dataset = tf.data.experimental.load(path + "tf_dataset")

# determine size of the dataset
dataset_size = sum(1 for _ in dataset)
print(f"dataset_size: {dataset_size}")

def lambda_1(features, labels, targets):
  return (features, labels)

def lambda_2(features, labels, targets):
  # features = features [:sample_length] 
  # targets = targets [:sample_length] 
  return (targets, features )  

# obtain the datasets containing only labels or targets
# label_dataset = dataset.map(lambda features, labels, targets: lambda_1(features, labels, targets))
target_dataset = dataset.map(lambda features, labels, targets: lambda_2(features, labels, targets))

# shuffle the dataset before splitting in train and validate!
# target_dataset = target_dataset.shuffle(dataset_size)

# split dataset in train and validate
train_fraction = 0.8
validate_dataset_size = int(dataset_size * (1.0-train_fraction)) # 20 percent of dataset_size
train_dataset = target_dataset.skip(validate_dataset_size)
validate_dataset = target_dataset.take(validate_dataset_size)

In [ ]:
import matplotlib.pyplot as plt
from IPython.display import Audio, display
import librosa
import numpy as np

for sample in train_dataset.shuffle(dataset_size).take(1):
  audio = sample[0][0].numpy().astype("float32") 
  label = sample[1].numpy()

  # print(label)
  # break
  print(audio.shape)
  plt.plot(audio)
  plt.title(f"Label: {label}")
  plt.show()
  plt.close()

  display(Audio(audio, rate=8000))

  mel = librosa.feature.melspectrogram(
      y=audio, n_mels=128, hop_length=64, sr=8000, fmax=2000
  )

  mel /= np.max(mel)
  plt.imshow(mel[::-1, :], cmap="inferno")

In [ ]:
from scipy.signal.spectral import spectrogram
import tensorflow as tf
import tensorflow_io as tfio
from keras.preprocessing.sequence import pad_sequences

def pad_second_dim(input, desired_size):
    padding = tf.tile([[0]], tf.stack([tf.shape(input)[0], desired_size - tf.shape(input)[1]], 0))
    return tf.concat([input, padding], 1)

def preprocess(sample, target):
  audio = sample[0]
  label = target[0]
  spectrogram = tfio.audio.spectrogram(audio, nfft=1024, window=1024, stride=64)
  spectrogram = tfio.audio.melscale(spectrogram, rate=8000, mels=128, fmin=0, fmax=2000)
  spectrogram /= tf.math.reduce_max(spectrogram)
  spectrogram = tf.expand_dims(spectrogram, axis=-1)

  spectrogram = tf.image.resize(spectrogram, (128, 128))
  spectrogram = tf.transpose(spectrogram, perm=(1, 0, 2))
  spectrogram = spectrogram[::-1, :, :]

  return spectrogram, label

def preprocess_both(sample, target):
  audio = sample[0]
  label = target[0]
  outs = []
  for audio in [audio, label]:
    spectrogram = tfio.audio.spectrogram(audio, nfft=1024, window=1024, stride=64)
    spectrogram = tfio.audio.melscale(spectrogram, rate=8000, mels=128, fmin=0, fmax=2000)
    spectrogram /= tf.math.reduce_max(spectrogram)
    spectrogram = tf.expand_dims(spectrogram, axis=-1)

    spectrogram = tf.image.resize(spectrogram, (128, 128))
    spectrogram = tf.transpose(spectrogram, perm=(1, 0, 2))
    spectrogram = spectrogram[::-1, :, :]
    outs.append(spectrogram)

  return outs[0], outs[1]

def preprocess3(sample, target):
  audio = sample[0]
  label = target[0]
  audio = audio [:sample_length] 
  label = label [:sample_length] 

  return audio, label


dataset_check = train_dataset.map(lambda sample, target: preprocess3(sample, target))

# for x, y in dataset_check.shuffle(dataset_size).take(4):
  # plt.imshow(x[:,:,0], cmap="inferno")
  # plt.plot(x)
  # reconstruction = tfio.audio.inverse_spectrogram((x[:,:,0], nfft=1024, window=1024, stride=64, iterations=30)
  # plt.plot(reconstruction)
  # display(Audio(reconstruction, rate=8000))


  # plt.title(f"{x.shape}, label: {y.numpy()}")
  # plt.show()
  # plt.close()

In [ ]:
dataset_train = train_dataset.map(lambda sample, target: preprocess3(target, sample))
# dataset_train = train_dataset
dataset_train = dataset_train.cache()
dataset_train = dataset_train.shuffle(dataset_size)
dataset_train = dataset_train.batch(32)

dataset_validate = validate_dataset.map(lambda sample, target: preprocess3(target, sample))
# dataset_validate = validate_dataset
dataset_validate = dataset_validate.cache()
dataset_validate = dataset_validate.batch(32)

In [ ]:
import ddsp

# Get synthesizer parameters from the input audio.
# outputs = network(audio_input)

# Initialize signal processors.
# harmonic = ddsp.synths.Harmonic()
# filtered_noise = ddsp.synths.FilteredNoise()
# reverb = ddsp.effects.TrainableReverb()
kwargs = {'return_outputs_dict': True}
reverb = ddsp.effects.Reverb(trainable=True, reverb_length=16000, add_dry=True, )
spectral_loss = ddsp.losses.SpectralLoss()

# Generate audio.
# audio_harmonic = harmonic(outputs['amplitudes'],
#                           outputs['harmonic_distribution'],
#                           outputs['f0_hz'])
# audio_noise = filtered_noise(outputs['magnitudes'])
# audio = audio_harmonic + audio_noise
# audio = reverb(audio)

# Multi-scale spectrogram reconstruction loss.
# loss = spectral_loss(audio, audio_input)

In [ ]:
import gin
reverb = ddsp.effects.Reverb(trainable=True, reverb_length=16000, add_dry=True, )

dag = [
  (reverb,
   ['add/signal']),
]

processor_group = ddsp.processors.ProcessorGroup(dag=dag)

In [ ]:
  def get_signal(audio):
    audio, ir = audio[0], audio[1]
    """Apply impulse response.
    Args:
      audio: Dry audio, 2-D Tensor of shape [batch, n_samples].
      ir: 3-D Tensor of shape [batch, ir_size, 1] or 2D Tensor of shape
        [batch, ir_size].
    Returns:
      tensor of shape [batch, n_samples]
    """
    # audio, ir = tf_float32(audio), tf_float32(ir)
    # ir = self._mask_dry_ir(ir)
    wet = ddsp.core.fft_convolve(audio, ir, padding='same', delay_compensation=0)
    return (wet + audio) #if self._add_dry else wet

In [ ]:
for x, y in train_dataset.take(1):
  print 
  x = reverb(x, x)


In [ ]:
from tensorflow.keras import models, layers
import tensorflow.keras as keras
conv1d_filters = 16
conv1d_strides = 3
my_input_shape = (128, 128, 1)

from tensorflow.python.keras.layers import Lambda;

def rev(x):
    x = ddsp.effects.Reverb(trainable=True, reverb_length=16000, add_dry=True,)(x)
    return x


# encoder2 = models.Sequential()
# model.add(layers.Input((32, 1, 64000)))
# encoder2.add(layers.Reshape((16000,)))
# encoder2.add(processor_group())
# encoder2.add(reverb(training=True) )
# encoder2.add(ddsp.effects.Reverb(trainable=True, reverb_length=16000, add_dry=True,))
# model.add(layers.Flatten())
# model.add(layers.Dense(3, activation="softmax"))
# encoder2.summary()

encoder_input = keras.Input(shape=(1, 16000), name="audio")
audio, ir = encoder_input, encoder_input
x = layers.Reshape((16000,))(audio)
audio = x
pack = (audio, ir)
# x = reverb(input)
# x = processor_group(x)
# x = layers.Conv2D(16, 3, activation="relu")(encoder_input)
# x = layers.Conv2D(32, 3, activation="relu")(x)
# x = layers.MaxPooling2D(3)(x)
# x = layers.Conv2D(32, 3, activation="relu")(x)
# x = layers.Conv2D(16, 3, activation="relu")(x)
# print(encoder_input.shape)
# x = reverb(x, return_outputs_dict=True, training=True )  # , return_outputs_dict=True   ["controls"] ["audio"]
# encoder_output = reverb(x, x, return_outputs_dict=True) ["controls"]
# wet = ddsp.core.fft_convolve(audio, ir, padding='same', delay_compensation=0)

x = Lambda(get_signal)(pack);
encoder_output = layers.Reshape((1, 16000))(x);
# encoder_output = ddsp.effects.Reverb(trainable=True, reverb_length=16000, add_dry=True,)(x)

encoder2 = keras.Model(encoder_input, encoder_output, name="encoder2" )
# encoder.summary()



encoder2.compile(
    optimizer="adam",
    loss = spectral_loss,
    # loss=tf.keras.losses.MeanSquaredError(),
    # loss="binary_crossentropy",
    # loss="sparse_categorical_crossentropy",
    # metrics=["accuracy"]
)

history = encoder2.fit(
    dataset_train,
    epochs=50,
    # validation_data=dataset_validate
)

In [ ]:
encoder_input = keras.Input(shape=(sample_length,), name="img")
audio = layers.Reshape((sample_length, ))(encoder_input)
xim = ddsp.spectral_ops.stft(audio)
xr = tf.math.real(xim)
xi = tf.math.imag(xim)
x = tf.stack((xr, xi), axis=-1)
x = layers.Reshape((32, 1025, 2))(x)
x = layers.Conv2D(16, 3, activation="relu")(x)
x = layers.Conv2D(32, 3, activation="relu")(x)
x = layers.MaxPooling2D(3)(x)
x = layers.Conv2D(32, 3, activation="relu")(x)
x = layers.Conv2D(16, 3, activation="relu")(x)
encoder_output = layers.GlobalMaxPooling2D()(x)

encoder = keras.Model(encoder_input, encoder_output, name="encoder")
encoder.summary()

x = layers.Reshape((4, 4, ))(encoder_output)
x = layers.Conv1DTranspose(16, 3, activation="relu")(x)
x = layers.UpSampling1D(10)(x)

x = layers.Conv1DTranspose(32, 3, activation="relu")(x)
x = layers.UpSampling1D(10)(x)
x = layers.Conv1DTranspose(16, 3, activation="relu")(x)
x = layers.UpSampling1D(10)(x)
x = layers.Conv1DTranspose(1, 3, activation="relu")(x)
x = layers.Reshape((6222, ))(x)
# x = layers.Flatten(x)
x = layers.Dense( sample_length)(x)
decoder_output = layers.Reshape((16000, ))(x + audio)

autoencoder = keras.Model(encoder_input, decoder_output, name="autoencoder")
autoencoder.summary()



autoencoder.compile(
    optimizer="adam",
    # loss=tf.keras.losses.MeanSquaredError(),
    loss=ddsp.losses.SpectralLoss()
    # loss="binary_crossentropy",
    # loss="sparse_categorical_crossentropy",
    # metrics=["accuracy"]
)

history = autoencoder.fit(
    dataset_train,
    epochs=50,
    validation_data=dataset_validate
)

In [ ]:
frame_length = 1024
overlap = 0.9
frame_step = int(frame_length * (1.0 - overlap))
window_fn = tf.signal.hamming_window



encoder_input = keras.Input(shape=(sample_length,), name="img")
audio = layers.Reshape((sample_length, ))(encoder_input)
# xim = ddsp.spectral_ops.stft(audio)
xim = tf.signal.stft(
    audio, frame_length, frame_step, window_fn=window_fn)
xr = tf.math.real(xim)
xi = tf.math.imag(xim)
xb = tf.stack((xr, xi), axis=-1)
x = layers.Reshape((147, 513, 2))(xb)
x = layers.Conv2D(16, 3, activation="relu")(x)
x = layers.Conv2D(32, 3, activation="relu")(x)
x = layers.MaxPooling2D(3)(x)
x = layers.Conv2D(32, 3, activation="relu")(x)
x = layers.Conv2D(16, 3, activation="relu")(x)
# encoder_output = layers.GlobalMaxPooling2D()(x)

encoder = keras.Model(encoder_input, encoder_output, name="encoder")
encoder.summary()
x = layers.Reshape((43, 165, 16))(encoder_output)
# x = layers.Reshape((4, 4, 1))(encoder_output)
x = layers.Conv2DTranspose(16, 3, activation="relu")(x)
# x = layers.UpSampling2D(10)(x)
x = layers.Conv2DTranspose(32, 3, activation="relu")(x)

x = layers.UpSampling2D(3)(x)
x = layers.Conv2DTranspose(16, 3, activation="relu")(x)
# x = layers.UpSampling2D(3)(x)
x = layers.Conv2DTranspose(1, 3, activation="relu")(x)
x = tf.dtypes.complex(
    x[:,:,0], x[:,:,1], name=None
)
x = tf.signal.inverse_stft(
    x, frame_length, frame_step, fft_length=None,
    window_fn=window_fn, name=None
)
# x = layers.Reshape((6222, ))(x)
# x = layers.Flatten(x)
x = layers.Dense( sample_length)(x)
decoder_output = layers.Reshape((16000, ))(x )  #  + audio

autoencoder = keras.Model(encoder_input, decoder_output, name="autoencoder")
autoencoder.summary()



autoencoder.compile(
    optimizer="adam",
    # loss=tf.keras.losses.MeanSquaredError(),
    loss=ddsp.losses.SpectralLoss()
    # loss="binary_crossentropy",
    # loss="sparse_categorical_crossentropy",
    # metrics=["accuracy"]
)

history = autoencoder.fit(
    dataset_train,
    epochs=50,
    validation_data=dataset_validate
)

In [ ]:
tf.keras.backend.clear_session()

frame_length = 512
overlap = 0.75
frame_step = int(frame_length * (1.0 - overlap))
window_fn = tf.signal.hamming_window



encoder_input = keras.Input(shape=(sample_length,), name="img")
audio = layers.Reshape((sample_length, ))(encoder_input)
# xim = ddsp.spectral_ops.stft(audio)
xim = tf.signal.stft(
    audio, frame_length, frame_step, window_fn=window_fn)
xr = tf.math.real(xim)
xi = tf.math.imag(xim)
# xb = tf.stack((xr, xi), axis=-1) # axis=-1
xb = tf.concat([tf.math.real(xim), tf.math.imag(xim)], axis=-1)
x = tf.expand_dims(xb, 3)
# x = layers.Reshape((28, 2050, 1))(xb)
x = layers.BatchNormalization()(x)
x = layers.Conv2D(16, 3, activation="relu")(x)
x = layers.Conv2D(32, 3, activation="relu")(x)
x = layers.MaxPooling2D(3)(x)
x = layers.Conv2D(32, 3, activation="relu")(x)
x = layers.Conv2D(16, 3, activation="relu")(x)
# encoder_output = layers.GlobalMaxPooling2D()(x)
encoder_output = x #xb

encoder = keras.Model(encoder_input, encoder_output, name="encoder")
encoder.summary()
x = layers.Reshape(encoder_output.shape[1:])(encoder_output)
# x = layers.Reshape((4, 678, 16))(encoder_output)
# x = layers.Reshape((4, 4, 1))(encoder_output)
x = layers.Conv2DTranspose(16, 3, activation="relu")(x)
# x = layers.UpSampling2D(10)(x)
x = layers.Conv2DTranspose(32, 3, activation="relu")(x)

x = layers.UpSampling2D(3)(x)
x = layers.Conv2DTranspose(16, 3, activation="relu")(x)
# x = layers.UpSampling2D(3)(x)
x = layers.Conv2DTranspose(1, 3, activation="tanh")(x)

# x = layers.Reshape((58, 2050 ))(x)
x = layers.Reshape((121, 514))(x)
# x = tf.squeeze(x)

# x = tf.dtypes.complex(
    # encoder_output[:,:,0], encoder_output[:,:,1], name=None
# )
x = tf.dtypes.complex(
    x[:, :, :257], x[:, :, 257:], name=None
)

x = tf.signal.inverse_stft(
    x, frame_length, frame_step,
    window_fn=tf.signal.inverse_stft_window_fn(
       frame_step, forward_window_fn=window_fn))

# x = layers.Reshape((6222, ))(x)
# x = layers.Flatten(x)
# x = tf.pad(x)
# x = layers.Dense( sample_length)(x)

paddings = [[0, 0], [0, sample_length-tf.shape(x)[1]]]
x = tf.pad(x, paddings, 'CONSTANT', constant_values=0)

# decoder_output = layers.Reshape((16000, ))(x )  #  + audio
decoder_output = x 

autoencoder = keras.Model(encoder_input, decoder_output, name="autoencoder")
autoencoder.summary()



autoencoder.compile(
    optimizer="adam",
    # loss=tf.keras.losses.MeanSquaredError(),
    loss=ddsp.losses.SpectralLoss()
    # loss="binary_crossentropy",
    # loss="sparse_categorical_crossentropy",
    # metrics=["accuracy"]
)

history = autoencoder.fit(
    dataset_train,
    epochs=50,
    validation_data=dataset_validate
)

In [ ]:
history = autoencoder.fit(
    dataset_train,
    epochs=50,
    validation_data=dataset_validate
)

In [ ]:
import matplotlib.pyplot as plt 
from IPython.display import Audio, display

sample_from_batch = 1

for audio, target in dataset_train.take(1):
  pred = autoencoder.predict(audio)[sample_from_batch]
  plt.plot(pred, alpha=0.6)
  # plt.show()
  # plt.close()
  plt.plot(target[sample_from_batch], alpha=0.6)
  plt.show()
  plt.close()
  display(Audio(pred, rate=8000))
  display(Audio(target[sample_from_batch], rate=8000))
  display(Audio(audio[sample_from_batch], rate=8000))
  print(ddsp.losses.SpectralLoss()(pred, target))



In [ ]:
encoder_input = keras.Input(shape=(1, sample_length,), name="img")
x = layers.Reshape((sample_length, 1))(encoder_input)
x = layers.Conv1D(16, 3, activation="relu")(x)
x = layers.Conv1D(32, 3, activation="relu")(x)
x = layers.MaxPooling1D(3)(x)
x = layers.Conv1D(32, 3, activation="relu")(x)
x = layers.Conv1D(16, 3, activation="relu")(x)
encoder_output = layers.GlobalMaxPooling1D()(x)

encoder = keras.Model(encoder_input, encoder_output, name="encoder")
encoder.summary()

x = layers.Reshape((4, 4, ))(encoder_output)
x = layers.Conv1DTranspose(16, 3, activation="relu")(x)
x = layers.Conv1DTranspose(32, 3, activation="relu")(x)
x = layers.UpSampling1D(3)(x)
x = layers.Conv1DTranspose(16, 3, activation="relu")(x)
x = layers.Conv1DTranspose(1, 3, activation="relu")(x)
x = layers.Flatten(x)
decoder_output = layers.Dense( sample_length)(x)

autoencoder = keras.Model(encoder_input, decoder_output, name="autoencoder")
autoencoder.summary()



autoencoder.compile(
    optimizer="adam",
    # loss=tf.keras.losses.MeanSquaredError(),
    loss=ddsp.losses.SpectralLoss()
    # loss="binary_crossentropy",
    # loss="sparse_categorical_crossentropy",
    # metrics=["accuracy"]
)

history = autoencoder.fit(
    dataset_train,
    epochs=50,
    # validation_data=dataset_validate
)

In [ ]:
from tensorflow.keras import models, layers

conv1d_filters = 16
conv1d_strides = 3
my_input_shape = (128, 128, 1)

model = models.Sequential()
model.add(layers.Conv2D(4, (3, 3), padding="same", activation="relu", input_shape=my_input_shape))
model.add(layers.MaxPool2D((2, 2)))

model.add(layers.Conv2D(8, (3, 3), padding="same", activation="relu", input_shape=my_input_shape))
model.add(layers.MaxPool2D((2, 2)))

model.add(layers.Conv2D(16, (3, 3), padding="same", activation="relu", input_shape=my_input_shape))
model.add(layers.MaxPool2D((2, 2)))

model.add(layers.Conv2D(32, (3, 3), padding="same", activation="relu", input_shape=my_input_shape))
model.add(layers.MaxPool2D((2, 2)))


model.add(layers.Flatten())
model.add(layers.Dense(3, activation="softmax"))
model.summary()

model.compile(
    optimizer="adam",
    # loss=tf.keras.losses.MeanSquaredError(),
    # loss="binary_crossentropy",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

history = model.fit(
    dataset_train,
    epochs=50,
    validation_data=dataset_validate
)

In [ ]:
def render_history(history):
    plt.plot(history["loss"], label="loss")
    plt.plot(history["val_loss"], label="val_loss")
    plt.legend()
    plt.title("Our losses")
    plt.show()
    plt.close()

    plt.plot(history["accuracy"], label="accuracy")
    plt.plot(history["val_accuracy"], label="val_accuracy")
    plt.legend()
    plt.title("Our accuracies")
    plt.show()
    plt.close()


def compare_histories():
    for training_name, history in history_list.items():
        plt.plot(history["val_accuracy"], label=training_name)
    plt.legend()
    plt.title("Comparision of val_accuracy")
    plt.show()
    plt.close()

render_history(history.history)